In [2]:

import time
from sklearn.svm import SVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_aer import AerSimulator, Aer
from qiskit.circuit.library import ZZFeatureMap
from Dataset.PreProcessing import caricaDataset


dataset = "breast_cancer"
n_dim, sample_train, label_train, sample_test, label_test = caricaDataset(dataset)


num_qubits = n_dim
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement='linear')    

# Create the quantum kernel
adhoc_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=Aer.get_backend('statevector_simulator'))

time_classic_kernel_pre = time.time()
time.sleep(1)
normal_svc = SVC(kernel="poly")
normal_svc.fit(sample_train, label_train)
y = normal_svc.score(sample_test, label_test)
time_classic_kernel_post = time.time()
y_time = time_classic_kernel_post - time_classic_kernel_pre


time_quantum_prekernel_pre = time.time()
time.sleep(1)
matrix_train = adhoc_kernel.evaluate(x_vec=sample_train)
matrix_test = adhoc_kernel.evaluate(x_vec=sample_test, y_vec=sample_train)
zzpc_svc = SVC(kernel='precomputed')
zzpc_svc.fit(matrix_train, label_train)
z = zzpc_svc.score(matrix_test, label_test)
time_quantum_prekernel_post = time.time()
z_time = time_quantum_prekernel_post - time_quantum_prekernel_pre

time_quantum_kernel_pre = time.time()
time.sleep(1)
quantum_svc = SVC(kernel=adhoc_kernel.evaluate)
quantum_svc.fit(sample_train, label_train)
x = quantum_svc.score(sample_test, label_test)
time_quantum_kernel_post = time.time()
x_time = time_quantum_kernel_post - time_quantum_kernel_pre

print(f'dataset: ' + dataset)
print(f'Precomputed kernel classification test score and time: {z,z_time} \n Callable kernel   classification test score: {x,x_time} \n normal svc test score : {y,y_time} \n')
print("\n")

dataset: breast_cancer
Precomputed kernel classification test score and time: (0.96, 7.427336931228638) 
 Callable kernel   classification test score: (0.96, 7.312180995941162) 
 normal svc test score : (0.96, 1.0010457038879395) 


